In [1]:
# Install packages from requirements.txt

# %pip install -r requirements.txt

# imports

In [1]:
# Set the working directory and adjust the Python path
import os
import sys

# Set the working directory to the src directory
os.chdir('./')
sys.path.append(os.getcwd())

from scripts.utils.generators import *
from scripts.test.test_graph_generators import *
from configuration.algorithm_settings import settings

from collections import Counter
from time import time
from scripts.utils.generators import generate_pheromone_map
import numpy as np
from scripts.utils.roulette_selection import roulette_wheel_selection

import pprint


# get graph map

In [2]:
size=10
fixed_weight=0.1

map_graph = generate_square_city_graph(size,fixed_weight)

In [3]:
buses_graph = generate_bus_line_square_city(size,fixed_weight)

In [4]:
map_graph = merge_bus_and_map_graph(map_graph,buses_graph)

# run ACO algorithms

## Ant best worst

In [14]:
start_node = 4
end_node = 52
num_ants = settings["ants"]
evaporation_rate =0.8
initial_pheromone_lvl =settings["f_ini"]
heuristic_weight=settings["alfa"]
pheromone_weight=settings["beta"]

epomax = settings["epomax"]
local_evap_rate=0.5
transition_prob=settings["transition_probability"]

solution_path,cost,time,epochs = ACS(map_graph, start_node, end_node, num_ants, evaporation_rate, local_evap_rate, transition_prob, initial_pheromone_lvl, heuristic_weight, pheromone_weight)

print(solution_path)
print(cost)
print(time)
print(epochs)

1
array([  inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf, 1.7  ,
       3.295,   inf,   inf, 1.9  ,   inf,   inf])
2
array([1.995, 1.5  , 1.3  ,   inf,   inf, 1.1  , 1.3  , 2.195, 2.7  ,
       2.1  , 0.9  , 5.195, 3.295,   inf, 1.5  ])
3
array([1.3  , 0.9  , 1.5  , 1.3  , 1.3  , 0.9  , 0.9  , 1.9  , 0.995,
       0.7  , 1.1  , 0.9  , 1.9  , 0.9  , 0.9  ])
4
array([1.3  , 1.1  , 1.3  , 1.9  ,   inf, 0.9  , 1.1  , 0.9  , 1.3  ,
       1.5  , 1.1  , 0.7  , 0.9  , 0.9  , 1.195])
5
array([0.9  , 0.7  , 0.9  , 1.9  , 0.9  , 1.5  , 1.3  , 1.1  , 1.3  ,
       1.3  , 1.3  , 1.3  , 0.7  , 0.995, 1.1  ])
6
array([1.1  , 1.1  , 1.9  , 0.7  , 0.9  , 1.1  , 0.7  , 1.3  , 2.1  ,
       1.9  , 1.7  , 1.9  , 1.3  , 0.995, 1.3  ])
7
array([1.9  , 0.9  , 1.1  , 1.1  , 1.1  , 0.9  , 1.5  , 0.9  , 1.5  ,
       0.995, 1.9  , 2.1  , 1.7  , 1.9  , 1.9  ])
8
array([0.9  , 0.9  , 0.995, 1.1  , 0.9  , 0.995, 1.1  , 0.9  , 0.9  ,
       1.3  , 1.3  , 1.3  , 0.995, 1.1  , 0.9  ])
9
array([1.3, 0.9, 0.9, 